# Document Classification with Naive Bayes - Lab

## Introduction

In this lesson, you'll practice implementing the Naive Bayes algorithm on your own.

## Objectives

In this lab you will:  

* Implement document classification using Naive Bayes

## Import the dataset

To start, import the dataset stored in the text file `'SMSSpamCollection'`.

In [1]:
# Your code here
import pandas as pd
df = pd.read_csv('SMSSpamCollection', sep='\t', names=['label', 'text'])
df.head()

,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [2]:
df.label.value_counts()

ham     4825
spam     747
Name: label, dtype: int64

## Account for class imbalance

To help your algorithm perform more accurately, subset the dataset so that the two classes are of equal size. To do this, keep all of the instances of the minority class (spam) and subset examples of the majority class (ham) to an equal number of examples.

In [4]:
# Your code here
minority= df[df['label']=='spam']
undersampled_majority = df[df['label']=='ham'].sample(n=len(minority))
df2= pd.concat([minority, undersampled_majority])
df2.label.value_counts()

spam    747
ham     747
Name: label, dtype: int64

In [10]:
p_classes = dict(df2['label'].value_counts(normalize=True))
p_classes

{'spam': 0.5, 'ham': 0.5}

In [11]:
df2.iloc[0]

label                                                 spam
text     Free entry in 2 a wkly comp to win FA Cup fina...
Name: 2, dtype: object

## Train-test split

Now implement a train-test split on the dataset: 

In [13]:
# Your code here
from sklearn.model_selection import train_test_split
X = df2['text']
y = df2['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=22, test_size=0.25)
train_df = pd.concat([X_train, y_train], axis=1)
test_df = pd.concat([X_test, y_test], axis=1)
train_df.head()

,text,label
3092,LORD OF THE RINGS:RETURN OF THE KING in store ...,spam
2585,Hi happy birthday. Hi hi hi hi hi hi hi,ham
5563,Ard 6 like dat lor.,ham
3544,I'm e person who's doing e sms survey...,ham
2856,Your gonna be the death if me. I'm gonna leave...,ham


## Create the word frequency dictionary for each class

Create a word frequency dictionary for each class: 

In [14]:
# Your code here
class_word_freq={}
classes= train_df['label'].unique()
for class_ in classes:
    temp_df=train_df[train_df.label== class_]
    bag={}
    for row in temp_df.index:
        doc=temp_df['text'][row]
        for word in doc.split():
            bag[word]= bag.get(word, 0)+1
            class_word_freq[class_]=bag

## Count the total corpus words
Calculate V, the total number of words in the corpus: 

In [15]:
# Your code here
vocabulary = set()
for text in train_df['text']:
    for word in text.split():
        vocabulary.add(word)
v=len(vocabulary)
v

5902

## Create a bag of words function

Before implementing the entire Naive Bayes algorithm, create a helper function `bag_it()` to create a bag of words representation from a document's text.

In [16]:
# Your code here
def bag_it(doc):
    bag ={}
    for word in doc.split():
        bag[word]=bag.get(word,0)+1
    return bag

## Implementing Naive Bayes

Now, implement a master function to build a naive Bayes classifier. Be sure to use the logarithmic probabilities to avoid underflow.

In [20]:
# Your code here
import numpy as np
def classify_doc(doc, class_word_freq, p_classes, V, return_posteriors=False):
    bag = bag_it(doc)
    classes = []
    posteriors = []
    for class_ in class_word_freq.keys():
        p=p_classes[class_]
        for word in bag.keys():
            num= bag[word]+1
            denom=class_word_freq[class_].get(word, 0)+V
            p *=(num/denom)
            classes.append(class_)
            posteriors.append(p)
        if return_posteriors:
            print(posteriors)
    return classes[np.argmax(posteriors)]


## Test your classifier

Finally, test your classifier and measure its accuracy. Don't be perturbed if your results are sub-par; industry use cases would require substantial additional preprocessing before implementing the algorithm in practice.

In [21]:
# Your code here
classify_doc(train_df.iloc[0]['text'], class_word_freq, p_classes, v, return_posteriors= True)

[0.0001694053870913095, 8.60799731155028e-08, 4.372501174170477e-11, 1.48145050793511e-14, 5.019313935067288e-18, 1.6877316526789805e-21, 5.7172481459315055e-25, 1.9370652705171964e-28, 9.842811333928844e-32, 3.319666554444804e-35, 1.6507541295100962e-38, 5.59293284604471e-42, 1.873365548834269e-45, 6.347164319275856e-49, 2.146487764381419e-52, 7.272531812235876e-56, 2.463177582467697e-59, 8.345511036651522e-63, 2.8127775654369807e-66, 9.481805378179608e-70, 2.981231057437387e-73, 1.0090475740184082e-76, 3.415868564720407e-80]
[0.0001694053870913095, 8.60799731155028e-08, 4.372501174170477e-11, 1.48145050793511e-14, 5.019313935067288e-18, 1.6877316526789805e-21, 5.7172481459315055e-25, 1.9370652705171964e-28, 9.842811333928844e-32, 3.319666554444804e-35, 1.6507541295100962e-38, 5.59293284604471e-42, 1.873365548834269e-45, 6.347164319275856e-49, 2.146487764381419e-52, 7.272531812235876e-56, 2.463177582467697e-59, 8.345511036651522e-63, 2.8127775654369807e-66, 9.481805378179608e-70, 2.98

'ham'

In [24]:
y_hat_train = X_train.map(lambda x: classify_doc(x, class_word_freq, p_classes, v))
residuals = y_train == y_hat_train
residuals.value_counts(normalize=True)

False    0.911607
True     0.088393
dtype: float64

## Level up (Optional)

Rework your code into an appropriate class structure so that you could easily implement the algorithm on any given dataset.

## Summary

Well done! In this lab, you practiced implementing Naive Bayes for document classification!